In [1]:
"""
模型参数检索
"""
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

##### 模型定义
##### 也可以直接使用 nn.functional 来定义网络层， 但是这种情况下该层参数并不会自动添加到模型 modules
class Simple_net(nn.Module):
    def __init__(self):
        super(Simple_net, self).__init__()
        
        self.conv0 = nn.Conv2d(10, 20, kernel_size=3, stride=1, padding=1)
        self.bn0 = nn.BatchNorm2d(20)
        self.relu0 = nn.PReLU()    
        
        self.conv1 = nn.Conv2d(20, 10, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(10)
        self.relu1 = nn.PReLU()
        
        self.pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(10, 2)
        
    def forward(self, inputs):
        output = self.relu0(self.bn0(self.conv0(inputs)))
        output = self.relu1(self.bn1(self.conv1(output)))
        output = self.fc(self.pool(output))
        
        return torch.sigmoid(output)

In [2]:
# 实例化模型同时构建优化器
model = Simple_net()
optimizer = optim.Adam(model.parameters(), lr=0.1)

In [5]:
# 模型 parameters 列表
para_list = list(model.parameters())                            # conv/fc 2; bn 1 
print('parameters list len: ', len(para_list))
print('first layer parameter: ', para_list[0].shape)            # 卷积层参数

parameters list len:  12
first layer parameter:  torch.Size([20, 10, 3, 3])


In [131]:
# 模型 named parameters 列表
named_para_list = list(model.named_parameters())
first_layer_para_name = named_para_list[0][0]            # conv0 name --> conv0.weight
first_layer_para_data = named_para_list[0][1]            # conv0 weight --> tensor

print('named parameters list len: ', len(named_para_list))
print('first layer param name: ', first_layer_para_name)
print('first layer param data: ', first_layer_para_data)

named parameters list len:  12
first layer param name:  conv0.weight
first layer param data:  Parameter containing:
tensor([[[[-0.0184, -0.0313, -0.0406],
          [-0.0637, -0.0931,  0.0688],
          [-0.0218,  0.0254, -0.0662]],

         [[ 0.0099,  0.0565,  0.0633],
          [-0.0497,  0.0920,  0.0997],
          [ 0.0769,  0.0816, -0.0015]],

         [[ 0.0769,  0.0079,  0.0538],
          [ 0.0684, -0.0090,  0.0540],
          [ 0.0089, -0.0913, -0.0649]],

         ...,

         [[ 0.0548, -0.0925,  0.0547],
          [-0.0083,  0.0603,  0.1018],
          [-0.0979, -0.0870,  0.0337]],

         [[-0.0712, -0.0302,  0.0966],
          [-0.0753,  0.0535, -0.1005],
          [ 0.0768,  0.0238, -0.0316]],

         [[-0.0633,  0.0743, -0.0443],
          [-0.0498, -0.0297, -0.0362],
          [ 0.0323,  0.0489, -0.0007]]],


        [[[-0.0138, -0.0691, -0.0992],
          [-0.0615,  0.0617, -0.0417],
          [-0.0045, -0.0429, -0.0068]],

         [[ 0.0704, -0.0688, -0.05

In [7]:
# 模型 modules 列表
# type(model.conv0) == nn.Conv2d
module_list = list(model.modules())
print('modules list len: ', len(module_list))

# 第一层卷积层
print('first module: ', module_list[1])

first_layer_param = list(module_list[1].parameters())   # 第一层卷积层参数， conv有两项参数，weight + bias
print('first_layer param number: ', len(first_layer_param),
      ' weights: ', first_layer_param[0].shape, 
      ' bias: ', first_layer_param[1].shape)

Conv2d(10, 20, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
modules list len:  9
first module:  Conv2d(10, 20, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
first_layer param number:  2  weights:  torch.Size([20, 10, 3, 3])  bias:  torch.Size([20])


In [123]:
# 模型 named modules 列表
named_module_list = list(model.named_modules())
print('named_list len: ', len(named_module_list))

# 第一层卷积
print('first named module: ', named_module_list[1])

first_layer_name = named_module_list[1][0]
first_layer_module = named_module_list[1][1]
first_layer_module_param = list(first_layer_module.parameters())
print('first layer name: ', first_layer_name)
print('first layer module: ', first_layer_module)
print('first layer module para: ', first_layer_module_param[0].shape, first_layer_module_param[1].shape)

named_list len:  9
first named module:  ('conv0', Conv2d(10, 20, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)))
first layer name:  conv0
first layer module:  Conv2d(10, 20, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
first layer module para:  torch.Size([20, 10, 3, 3]) torch.Size([20])


In [162]:
# 模型 state dict
model_state_dict = model.state_dict()
for key, value in model_state_dict.items():
    print(key, '             value shape: ', value.shape)

conv0.weight              value shape:  torch.Size([20, 10, 3, 3])
conv0.bias              value shape:  torch.Size([20])
bn0.weight              value shape:  torch.Size([20])
bn0.bias              value shape:  torch.Size([20])
bn0.running_mean              value shape:  torch.Size([20])
bn0.running_var              value shape:  torch.Size([20])
bn0.num_batches_tracked              value shape:  torch.Size([])
relu0.weight              value shape:  torch.Size([1])
conv1.weight              value shape:  torch.Size([10, 20, 3, 3])
conv1.bias              value shape:  torch.Size([10])
bn1.weight              value shape:  torch.Size([10])
bn1.bias              value shape:  torch.Size([10])
bn1.running_mean              value shape:  torch.Size([10])
bn1.running_var              value shape:  torch.Size([10])
bn1.num_batches_tracked              value shape:  torch.Size([])
relu1.weight              value shape:  torch.Size([1])
fc.weight              value shape:  torch.Size([2, 10

In [154]:
model_state_dict['conv0.weight'].shape

torch.Size([20, 10, 3, 3])

In [173]:
"""
初始化网络参数
model.apply(fn), 对网络中间层进行参数初始化
"""
net = nn.Sequential(nn.Linear(10, 2))
def init_weight(layer):
    if type(layer) == nn.Linear:
        layer.weight.data.fill_(1.0)

net.apply(init_weight)

Sequential(
  (0): Linear(in_features=10, out_features=2, bias=True)
)

In [ ]:
"""
torch 模型保存有两种方式（https://pytorch.org/docs/master/notes/serialization.html#recommend-saving-models）
第一种：只保存模型参数（state_dict）， 官方比较推荐这一种方式
第二种：保存整个模型

直接保存整个模型的话，其存储数据结构被限定特定格式，不利于后续的模型加载重用，比如当模型定义类发生变化时，加载模型可能会出问题
"""

#第一种：只保存模型参数
torch.save(model.state_dict(), save_path)  # save model
model = Model_class(**args)                # construct new model
model = model.load_state_dict(torch.load(save_path))  # load saved parameters to the model

#第二种：保存整个模型
torch.save(model, save_path)     # save the entire model
model = torch.load(save_path)    # load the model


In [12]:
'''
optimizer通过param_group来管理参数组.param_group中保存了参数组及其对应的学习率,动量等等.
所以我们可以通过更改param_group[‘lr’]的值来更改对应参数组的学习率。
'''
# 学习率衰减
def adjust_learning_rate(optimizer, epoch):
    """Sets the learning rate to the initial LR decayed by 10 every 30 epochs"""
    lr = args.lr * (0.1 ** (epoch // 30))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr
